## Notebook to perform correlations between estimated cell-type fractions in bulk RNAB and ATAC peaks and DNA methylation sites that are <i>cis</i> to monogenic risk genes

Use a GLMM to handle repeated measure from multiple days

Even though CHST is merge of ATAC and METH, need to load the seperate data as METH doesn't include da25 so neither does the merge but interested in seeing that here

In [ ]:
!date

#### import libraries

In [ ]:
from pandas import read_hdf, read_csv, DataFrame, concat
import concurrent.futures
import numpy as np
import statsmodels.api as sm
import statsmodels.stats.multitest as smm

import warnings
warnings.simplefilter('ignore')

#### set notebook variables

In [ ]:
# parameters
gene = '' # 'GBA', 'SNCA', or 'LRRK2'
gene_id = '' # 'ENSG00000177628.15', 'ENSG00000145335.15', or 'ENSG00000188906.15'
modality = '' # 'ATAC' or 'METH'

In [ ]:
# naming
cohort = 'foundin'
day = 'daALL'
feature_type = 'scaled.adj'

# directories
wrk_dir = '/labshare/raph/datasets/foundin_qtl'
quants_dir = f'{wrk_dir}/quants'
info_dir = f'{wrk_dir}/sample_info'
results_dir = f'{wrk_dir}/results'

# in files
quants_file = f'{quants_dir}/{cohort}_{day}_{modality}.{feature_type}.hdf5'
info_file = f'{info_dir}/{cohort}_{modality}_sample_info.csv'
rnab_file  = f'{quants_dir}/{cohort}_{day}_RNAB.{feature_type}.hdf5'

# out files
results_file = f'{results_dir}/{cohort}_{modality}_{gene}_cis_features_cell_fractions.csv'

# variables
DEBUG = False
# here excluding da0 as diff in cell-fractions at day 0 is just variation and error
days_to_include = ['da25', 'da65']
# here only interested in dopamengic or immature dopamenergic, or their sum
cell_types = ['DopaminergicNeurons', 'ImmatureDopaminergicNeurons', 'DAn', 
              'TH_Pel-Freez_ICC', 'MAP2_Santa_Cruz_ICC']
# monogenic regions: chrom, start, stop
gene_tuples = {'GBA1': [1, 153925709, 156235843], 
               'SNCA': [4, 88704960, 90715479], 
               'LRRK2': [12, 39220632, 41340400],
               'PINK1': [1, 20133458, 21133458],
               'PRKN': [6, 162227775, 163227775],
               'PARK7': [1, 7454291, 8454291],
               'VPS35': [16, 46189518, 47189518],
               'RAB39B': ['X', 154764491, 155764491],
               'GCH1': [14, 54402826, 55402826],
               'VPS13C': [15, 61560473, 62560473],
               'TAF1': ['X', 70866222, 71866222],
               'DAGLB': [7, 5984190, 6984190],
               'ALDH1A1': [9, 72580442, 73580442]               
              }
other_terms = ['sex', 'Batch', 'PC1', 'PC2', 'PC3']
MDL_OTHER_TERMS = '+ C(sex) + C(Batch) + PC1 + PC2 + PC3'
# other_terms = ['day', 'sex', 'Batch', 'PC1', 'PC2', 'PC3']
# MDL_OTHER_TERMS = '+ C(day) + C(sex) + C(Batch) + PC1 + PC2 + PC3'
alpha = 0.05

#### analysis functions

In [ ]:
def mixed_model(formula: str, df: DataFrame, group_name: str) -> list:
    model = sm.MixedLM.from_formula(formula, df, groups=df[group_name])
    result = model.fit()
    return result

def glmm(df: DataFrame, endo: str, exo: str, verbose: bool=False) -> tuple:
    grouping = 'sampleid'
    this_formula = f'Q("{endo}") ~ {exo} {MDL_OTHER_TERMS}'
    try:
        # run GLMM via statsmodel
        result = mixed_model(this_formula, df, grouping)
        ret_list = [endo, exo, result.params['Intercept'], 
                    result.params[exo], result.bse[exo], 
                    result.tvalues[exo], result.pvalues[exo]]
        if verbose:
            print(this_formula)
            print(result.summary())
            print(['endo', 'exo', 'intercept', 'coef', 'stderr', 'z', 'p-value'])
            print(ret_list)
    except:
#         print(f'Caught Error for {dep_term}')
        ret_list = [endo] + [exo] + [np.nan] * 5
        if verbose:
            print(this_formula)
            print(result.summary())
            print(['endo', 'exo', 'intercept', 'coef', 'stderr', 'z', 'p-value'])
            print(ret_list)    
  
    return ret_list

# compute B&H FDR for given p-values
def compute_fdr(pvalues):
    bh_adj = smm.fdrcorrection(pvalues)
    return bh_adj[1]

### load input data

#### load the sample info file, this contains the estimated cell fractions

In [ ]:
info_df = read_csv(info_file)
print(f'shape of info df is {info_df.shape}')
# subet to just samples from specified days
info_df = info_df.loc[info_df.day.isin(days_to_include)]
print(f'shape of info df for days {days_to_include} is {info_df.shape}')
if DEBUG:
    display(info_df.sample(5))
    display(info_df.day.value_counts())

#### load the quantied modality's features

In [ ]:
%%time
quants_df = read_hdf(quants_file)
print(f'shape of quants df is {quants_df.shape}')
# subet to just samples from specified days
quants_df = quants_df.loc[quants_df.index.isin(info_df.assayid)]
print(f'shape of quants df for days {days_to_include} is {quants_df.shape}')
if DEBUG:
    display(quants_df.sample(5))

### find modality's features that are <i>cis</i> to the target gene

In [ ]:
gene_tuple = gene_tuples.get(gene)
chrom = gene_tuple[0]
start_bp = gene_tuple[1]
stop_bp = gene_tuple[2]

print(f'{gene} region {chrom}:{start_bp}-{stop_bp}')

In [ ]:
if modality == 'ATAC':
    # if ATAC feature name is genomic coordinates
    peak_info = quants_df.columns.str.split('_', expand=True).to_frame()
    features_df = DataFrame(data={'feature': quants_df.columns})
    features_df['chrom'] = peak_info[0].values
    features_df['start'] = peak_info[1].astype(int).values
    features_df['stop'] = peak_info[2].astype(int).values
elif modality == 'METH':
    # if METH need to read in coords
    features_file = f'{quants_dir}/EPIC_annotation_hg38.txt'
    features_df = read_csv(features_file, sep='\t', header=None)
    features_df.columns = ['chrom', 'start', 'stop', 'feature']
    # for bed to work start != stop
    features_df.stop = features_df.start + 1

print(f'shape of full features {features_df.shape}')
# subset to features that are in the cis region
features_df = features_df.loc[(features_df.chrom == f'chr{chrom}') & 
                              (features_df.start >= start_bp) & 
                              (features_df.stop <= stop_bp)]
print(f'shape of region features {features_df.shape}')
if DEBUG:
    display(features_df.sample(5))

### subset the quantifications to just features in the region

In [ ]:
quants_df = quants_df[quants_df.columns.intersection(features_df.feature)]
print(f'shape of quants df region features {quants_df.shape}')
if DEBUG:
    display(quants_df.sample(5))

### load the quantied gene of interest

In [ ]:
%%time
rna_df = read_hdf(rnab_file)
print(f'shape of rna_df df is {rna_df.shape}')
# need to convert assay ID to diff modality for merging
rna_df['assayid'] = rna_df.index.str.replace('RNAB_', f'{modality}_')
rna_df = rna_df.set_index('assayid')
# subet to just samples from specified days
rna_df = rna_df.loc[rna_df.index.isin(info_df.assayid)]
# subset to just the gene of interest
rna_df = rna_df[[gene_id]]
print(f'shape of rna_df df for days {days_to_include} is {rna_df.shape}')
if DEBUG:
    display(rna_df.sample(5))

### merge region's quantified features with sample info

In [ ]:
info_terms = ['assayid', 'sampleid'] + cell_types + other_terms
input_df = quants_df.merge(info_df[info_terms], how='inner', 
                           left_index=True, right_on='assayid')
input_df = input_df.merge(rna_df, how='inner', 
                          left_on='assayid', right_index=True)
# # exclude any samples with missing input
input_df = input_df.loc[~input_df.isnull().any(axis='columns')]
print(f'model input data shape {input_df.shape}')
if DEBUG:
    display(input_df.sample(5))

### regress the cis features and cell fractions

In [ ]:
%%time

fs_list = []
lm_results = []
with concurrent.futures.ProcessPoolExecutor() as ppe:
# with concurrent.futures.ThreadPoolExecutor() as ppe:
    for cell_type in cell_types:
        for feature in features_df.feature:
            fs_list.append(ppe.submit(glmm, input_df, cell_type, feature))

In [ ]:
for future in concurrent.futures.as_completed(fs_list):
    lm_results.append(future.result())

# convert list of results to dataframe
results_df = DataFrame(lm_results, columns=['endo', 'exo', 'intercept', 'coef', 
                                            'stderr', 'z', 'p-value'])
print(f'shape of results df is {results_df.shape}')
if DEBUG:
    display(results_df.sample(5))

### apply B&H FDR corrections to results

In [ ]:
results_df['bh_fdr'] = compute_fdr(results_df['p-value'].fillna(1))
print(f'shape of significant results {results_df.loc[results_df.bh_fdr <= alpha].shape}')

if DEBUG:
    display(results_df.sample(5))

### save the results

In [ ]:
results_df.to_csv(results_file)

In [ ]:
temp = results_df.sort_values('p-value')
display(temp.head())

In [ ]:
glmm(input_df, temp.endo.values[0], temp.exo.values[0], verbose=DEBUG)

### model the monogenic gene against the cell fractions

the monogenics are already in the info files as targets for prediction models

In [ ]:
for cell_type in cell_types:
    print(cell_type)
    grouping = 'sampleid'
    this_formula = f'Q("{cell_type}") ~ Q("{gene_id}") {MDL_OTHER_TERMS}'    
    # run GLMM via statsmodel
    result = mixed_model(this_formula, input_df, grouping)
    print(this_formula)
    print(result.summary())
    this_exo = f'Q("{gene_id}")'
    print(f'coef: {result.params[this_exo]}, p-value: {result.pvalues[this_exo]}')

In [ ]:
!date